**1. Create a unigram inverted index(from scratch; No library allowed) of the dataset obtained
from Q1 (after preprocessing).** 

In [4]:

data_structure = {}           #using dictionary as a datastructure
file_list=[]                  #stores all file names ===> will be sued during OR NOT operation
for i in range(1, 1401):      #There are a total of 1400 files to access

  fname = "cranfield"         #file name starts with "cranfield"

  #Filename consists of file number. Since the number if of 4 digit we add corresponding zeros in the begining
  if(i>=1 and i<10):
    fname = fname + "000" + str(i)
  elif(i>=10 and i<100):
    fname = fname + "00" + str(i)
  elif(i>=100 and i<1000):
    fname = fname + "0" + str(i)
  else:
    fname = fname + str(i)

  file_list.append(fname)

  with open(fname) as f:       #open the corresponding file
    while True:
        l = f.readline()        #read line by line
        if not l:               #if line dosent exists means its EOF, hence get out of loop
            break
        else:
          for w in l.split():   #for every word in that line

            if w in data_structure.keys():    #check if that word exists in the data structure already
              if fname not in data_structure[w]:
                data_structure[w].append(fname)      #if it exists then just add the document name to the value
            else:
                data_structure[w] = [fname]        #else add the word as a key and document name as the value for that key
  

#displaying the datastructure for reference
for key in data_structure.keys():
  print(key, '->', data_structure[key])


Streaming output truncated to the last 5000 lines.
581 -> ['cranfield0245']
referred -> ['cranfield0245', 'cranfield0353', 'cranfield0414', 'cranfield0490', 'cranfield0658', 'cranfield0756', 'cranfield0813', 'cranfield0873']
hits -> ['cranfield0245']
takeoff -> ['cranfield0245', 'cranfield0416', 'cranfield0911', 'cranfield1064', 'cranfield1094', 'cranfield1095', 'cranfield1162', 'cranfield1163', 'cranfield1164']
flapped -> ['cranfield0245']
conformal -> ['cranfield0246', 'cranfield0432', 'cranfield0452', 'cranfield1197', 'cranfield1208', 'cranfield1223']
trefitz -> ['cranfield0246']
unequal -> ['cranfield0246', 'cranfield1119']
circulation -> ['cranfield0246', 'cranfield0363', 'cranfield0404', 'cranfield0433', 'cranfield0443', 'cranfield0452', 'cranfield0753', 'cranfield0927', 'cranfield1168', 'cranfield1330', 'cranfield1339']
theta -> ['cranfield0246']
setting -> ['cranfield0246', 'cranfield0392', 'cranfield0520', 'cranfield0677', 'cranfield0707']
ordinates -> ['cranfield0247', 'cranf

In [5]:
print(len(file_list))           #to check if all the files are present or not

1400


Reference:

https://www.pythontutorial.net/python-basics/python-read-text-file/

**2. Use Python’s pickle module to save and load the unigram inverted index.**

In [6]:
import pickle         

f = open("result", "x")               #create a file named result to store our data structure

with open('result', 'wb') as f:
  pickle.dump(data_structure, f)      # dump method is used to save variables (data structure we created in our case) to a pickle file.

In [7]:
with open('result', 'rb') as f:
   res = pickle.load(f)               # load method is used to load the contents of a pickle file

   print(res)

{'experimental': ['cranfield0001', 'cranfield0011', 'cranfield0012', 'cranfield0017', 'cranfield0019', 'cranfield0025', 'cranfield0029', 'cranfield0030', 'cranfield0035', 'cranfield0041', 'cranfield0047', 'cranfield0052', 'cranfield0053', 'cranfield0058', 'cranfield0069', 'cranfield0070', 'cranfield0074', 'cranfield0078', 'cranfield0084', 'cranfield0099', 'cranfield0101', 'cranfield0103', 'cranfield0112', 'cranfield0115', 'cranfield0121', 'cranfield0123', 'cranfield0137', 'cranfield0140', 'cranfield0142', 'cranfield0154', 'cranfield0156', 'cranfield0168', 'cranfield0170', 'cranfield0171', 'cranfield0173', 'cranfield0176', 'cranfield0179', 'cranfield0183', 'cranfield0184', 'cranfield0186', 'cranfield0187', 'cranfield0188', 'cranfield0189', 'cranfield0191', 'cranfield0195', 'cranfield0197', 'cranfield0202', 'cranfield0203', 'cranfield0206', 'cranfield0207', 'cranfield0212', 'cranfield0216', 'cranfield0220', 'cranfield0222', 'cranfield0225', 'cranfield0227', 'cranfield0230', 'cranfield023

References:

https://www.geeksforgeeks.org/how-to-use-pickle-to-save-and-load-variables-in-python/


**3. Provide support for the following operations:**

**a. T1 AND T2**

**b. T1 AND NOT T2**

**c. T1 OR T2**

**d. T1 OR NOT T2**

In [9]:
#preprocessing
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword = stopwords.words('english')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
#this function is sued to perform preprocssing over the input query data

def preprocess(sequence):
  sequence = sequence.lower()           #convert to lowercase
  sequence = re.sub(r'[^\w\s]','',sequence)     #remove punctuation
  sequence = nltk.word_tokenize(sequence)         #perform tokenization
  rem_stop = [s_word for s_word in sequence if s_word not in stopword]      #remove stopwords
  sequence  = ' '.join(str(token) for token in rem_stop)
  sequence = sequence.replace("  "," ")         #remove extra space

  return sequence

In [11]:
#to find minimum number of comparsions for AND operation and obtain the resultant list of files

def minComparisonAND(l1, l2, count):
  res = []      #stores final result of the operation
  
  len1 = len(l1)      #find length of list 1
  len2 = len(l2)      #find length of list 2

  if(len1<len2):      #keep larger list in l1 and smaller one in l2
    l1,l2 = l2,l1
    len1 = len(l1)
    len2 = len(l2)
 
  i=0       #to track elements in list l2
  j=0       #to track elements in list l1

  #for every element in list l1
  #if element from l1 = element from l2 ====> then add that element in final result and increment the count by 1, inc pointer to l2
  #if element from l1 > element from l2 ====> then inc pointer to l2 and go to next element
  #                 but we need current element of list l1 again so dec its pointer coz it will be incremented in the next iteration
  #                 incremenet count by 2 since this is 2nd comparsion
  #if above conditions fail means element from l1 < element from l and we can just ignore. Inc count by 2 since we have already done 2 comparsions
  while j<len1:
    if i==len2:
      break
    if l1[j]==l2[i]:
      res.append(l2[i])
      i=i+1
      count=count+1

    elif l1[j]>l2[i]:         #since lists are sorted if we go accross element which is greater than current then current element wont occur in the list and we can now move ahead
      i=i+1
      j=j-1
      count=count+2
    else:
      count = count+2

    j=j+1

  return res,count


a, b = minComparisonAND(['cranfield0777', 'cranfield1291', 'cranfield1344'], ['cranfield0045', 'cranfield0046', 'cranfield1291', 'cranfield1344'], 0)
print(a,b)

['cranfield1291', 'cranfield1344'] 8


In [26]:
def minComparsionANDNOT(l1, l2, count):
  i=0
  j=0
  res = []
  while i!=len(l1) and j!=len(l2):       #run untill we reach to the end of any one of the list
    if l2[j]>l1[i]:                      #if element of l2 > element of l1 ===> append element of l2 in the result
      res.append(l1[i])
      i = i+1
      count +=1 
    elif l2[j]<l1[i]:                    #if element of l2 < element of l1 ===> ignore l2 element
      j = j+1 
      count +=2 
    else:                                #if element of l2 = element of l1 ===> ignore both l1 and l2 element
      j=j+1
      i = i+1  
      count +=2  

#append remaining elements of list l1 to the result
  while i!=len(l1):
    res.append(l1[i])
    i = i+1

  return res,count

a, b = minComparsionANDNOT(['cranfield0045', 'cranfield0046', 'cranfield1291', 'cranfield1354'], ['cranfield0777', 'cranfield1291', 'cranfield1344'], 0)
print(a,b)

['cranfield0045', 'cranfield0046', 'cranfield1354'] 8


In [18]:
#standard logic to merge 2 sorted lists
def minComparisonOR(l1, l2, count):
  res = []
  
  len1 = len(l1)
  len2 = len(l2)

  #if anyone of the list is empty then return the other list
  if len1==0:
    return l2,count
  if len2==0:
    return l1, count
  i=0     #keeps track of list l1
  j=0     #keeps track of list l2

  while i<len1 and j<len2:        #run untill we reach to the end of any one of the list and append the smaller of the 2 elements to the list
    if l1[i]<l2[j]:
      res.append(l1[i])
      count = count+1
      i=i+1
    elif l1[i]>l2[j]:
      res.append(l2[j])
      count = count+2
      j=j+1
    else:
      res.append(l2[j])       #if both elements are same then append only once and increment both list pointers
      count = count+2

      i=i+1
      j=j+1
  
  if i == len1 and j ==len2:      #if we have reached end of both the lists then return
    return res,count

  if i == len1:             #if we are end of list l1 then add remaining elements of list l2 to the result
    while j<len2:
      res.append(l2[j])
      j =j+1

  elif j == len2:             #if we are end of list l2 then add remaining elements of list l1 to the result
    while i<len1:
      res.append(l1[i])
      i =i+1

  return res,count


a, b = minComparisonOR(['cranfield0777', 'cranfield1291', 'cranfield1344'], ['cranfield0045', 'cranfield0046', 'cranfield1291', 'cranfield1354'], 0)
print(a,b)

['cranfield0045', 'cranfield0046', 'cranfield0777', 'cranfield1291', 'cranfield1344', 'cranfield1354'] 8


In [28]:
def minComparisonORNOT(key1, key2, count, file_list):
  res=[]
  res,count = minComparisonOR(key1, key2, count)            #perform OR between 2 keys
  res,count = minComparsionANDNOT(file_list, key1, count)   #perform and not between result and universal list of all file names

  return res, count

a, b = minComparisonORNOT(['cranfield0777', 'cranfield1291', 'cranfield1344'], ['cranfield0045', 'cranfield0046', 'cranfield1291', 'cranfield1354'], 0, file_list)
print(a,b)

['cranfield0001', 'cranfield0002', 'cranfield0003', 'cranfield0004', 'cranfield0005', 'cranfield0006', 'cranfield0007', 'cranfield0008', 'cranfield0009', 'cranfield0010', 'cranfield0011', 'cranfield0012', 'cranfield0013', 'cranfield0014', 'cranfield0015', 'cranfield0016', 'cranfield0017', 'cranfield0018', 'cranfield0019', 'cranfield0020', 'cranfield0021', 'cranfield0022', 'cranfield0023', 'cranfield0024', 'cranfield0025', 'cranfield0026', 'cranfield0027', 'cranfield0028', 'cranfield0029', 'cranfield0030', 'cranfield0031', 'cranfield0032', 'cranfield0033', 'cranfield0034', 'cranfield0035', 'cranfield0036', 'cranfield0037', 'cranfield0038', 'cranfield0039', 'cranfield0040', 'cranfield0041', 'cranfield0042', 'cranfield0043', 'cranfield0044', 'cranfield0045', 'cranfield0046', 'cranfield0047', 'cranfield0048', 'cranfield0049', 'cranfield0050', 'cranfield0051', 'cranfield0052', 'cranfield0053', 'cranfield0054', 'cranfield0055', 'cranfield0056', 'cranfield0057', 'cranfield0058', 'cranfield005

In [29]:

ip = int(input("Enter number of queries to execute ===> "))           #take number of queries as input

sqlist=[]         #stores all the input sequences in a list
operlist = []     # stores operations to be performed for quesries

for i in range(1, ip+1):

  #take queries as input, perform preprocessing and add into sequence list one by one
  sequence = input("Enter Input sequence for query " + str(i) + " ===> ")
  temp = preprocess(sequence)
  ip_sequence = temp.split()    #consists sequence of one query
  sqlist.append(ip_sequence)

  #take operations as input and add into operations list one by one
  operations = input("Enter Operations separated by comma for query " + str(i) + " ===> ")
  op = operations.split(", ")       #consists operations of one query
  operlist.append(op)
  
#perform respective operations for every query  
for i in range(0, ip):

  res = []      #this is the resultant list

  flag=0            #to keep track of first iteration
  
  #pick one query and respective operation and add them to l1 and l2 respectively
  l1 = sqlist[i]
  l2 = operlist[i]
  
  count=0         #keeps track of number of comparisons
  #iterate through current subsequence
  for j in range(0, len(l1)):
    key1 = res                                    # reuse previous result into next operation
    key2 = data_structure[l1[j]]         # take next word from sequence

    # if flag=0 means its first iteration do OR with first key and reuse it
    # if flag=1 means its not inital iteration and we can now use corresponsing operations
    #op1 consists of current operation to be performed
    if(flag==0):
      op1 = "OR"
      flag=1
    else:
      op1 = l2[j-1]

    #perform operation as per the match found
    if(op1 == "AND"):
      res,count = minComparisonAND(key1, key2, count)
    elif(op1 == "OR"):
      res,count = minComparisonOR(key1, key2, count)
    elif(op1 == "AND NOT"):
      res,count = minComparsionANDNOT(key1, key2, count)
    elif(op1 == "OR NOT"):
      res,count = minComparisonORNOT(key1, key2, count, file_list)

  #display the result
  print("Number of documents retrieved for query " + str(i+1) +"= " + str(len(res)))
  print("Names of the documents retrieved for query " + str(i+1) + "= " + str(res))
  print("Number of comparisons required for query " +str(i+1) + "= " + str(count))


Enter number of queries to execute ===> 1
Enter Input sequence for query 1 ===> reinforced  is , the aluminumalloy  of ,,, HEADS
Enter Operations separated by comma for query 1 ===> OR, AND
Number of documents retrieved for query 1= 1
Names of the documents retrieved for query 1= ['cranfield0733']
Number of comparisons required for query 1= 41


In [ ]:
# sample examples
# Enter Input sequence for query 1 ===> reinforced  is , the aluminumalloy  of ,,, HEADS
# Enter Operations separated by comma for query 1 ===> OR, AND